# 25차시: 자연어 처리(NLP) 기초 - 토큰화와 임베딩

## 학습 목표
- 자연어 처리(NLP)의 기본 개념 이해
- 토큰화 방법 학습 (KoNLPy, tiktoken)
- TF-IDF로 텍스트 중요도 분석
- 임베딩과 유사도 검색 이해

## 학습 내용
1. NLP란 무엇인가?
2. 토큰화 (Tokenization)
3. TF-IDF
4. 임베딩과 유사도 검색

In [1]:
!pip install -Uq konlpy tiktoken langchain-huggingface sentence-transformers koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 16.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
from IPython.display import display

---
## 1. NLP란 무엇인가?

### 정의
**자연어 처리(Natural Language Processing, NLP)**: 컴퓨터가 인간의 언어를 이해하고 처리하는 기술

### 금융에서의 NLP 활용
| 활용 분야 | 설명 | 예시 |
|-----------|------|------|
| 감성 분석 | 텍스트의 긍/부정 판단 | 뉴스 감성 → 주가 영향 예측 |
| 정보 추출 | 핵심 정보 자동 추출 | 재무제표에서 수치 추출 |
| 텍스트 분류 | 문서 자동 분류 | 공시 유형 분류 |
| 요약 | 긴 문서 요약 | 애널리스트 리포트 요약 |

### NLP 파이프라인
```
원본 텍스트 → 토큰화 → 벡터화(임베딩) → 모델 학습/검색
```

---
## 2. 토큰화 (Tokenization)

### 정의
**토큰화**: 텍스트를 작은 단위(토큰)로 분리하는 작업

### 토큰화 방식
| 방식 | 설명 | 예시 |
|------|------|------|
| 공백 기반 | 띄어쓰기로 분리 | "삼성전자 주가" → ["삼성전자", "주가"] |
| 형태소 분석 | 의미 단위로 분리 (한국어) | "삼성전자가" → ["삼성전자", "가"] |
| BPE | 자주 등장하는 문자쌍 병합 | GPT 모델에서 사용 |

In [3]:
# 예시 금융 텍스트
sample_texts = [
    "삼성전자, 3분기 실적 발표 영업이익 10조원 돌파",
    "코스피 상승세 지속, 외국인 순매수 2000억원",
    "미국 연준 금리 0.25%p 인상 결정",
    "SK하이닉스 반도체 수출 호조 주가 5% 급등",
    "원달러 환율 1350원 돌파, 경제 불확실성 확대"
]

### 2.1 공백 기반 토큰화

In [4]:
# 간단한 공백 기반 토큰화
def simple_tokenize(text):
    """공백 기반 토큰화"""
    return text.split()

print("[공백 기반 토큰화]")
print("=" * 60)
for text in sample_texts[:3]:
    tokens = simple_tokenize(text)
    print(f"텍스트: {text}")
    print(f"토큰: {tokens}")
    print(f"토큰 수: {len(tokens)}")
    print()

[공백 기반 토큰화]
텍스트: 삼성전자, 3분기 실적 발표 영업이익 10조원 돌파
토큰: ['삼성전자,', '3분기', '실적', '발표', '영업이익', '10조원', '돌파']
토큰 수: 7

텍스트: 코스피 상승세 지속, 외국인 순매수 2000억원
토큰: ['코스피', '상승세', '지속,', '외국인', '순매수', '2000억원']
토큰 수: 6

텍스트: 미국 연준 금리 0.25%p 인상 결정
토큰: ['미국', '연준', '금리', '0.25%p', '인상', '결정']
토큰 수: 6



### 2.2 KoNLPy - 한국어 형태소 분석

한국어는 조사, 어미 등이 붙어서 공백 기반으로는 정확한 분석이 어렵습니다.
**형태소 분석기**를 사용하면 의미 단위로 정확히 분리할 수 있습니다.

In [5]:
# 한국어 형태소 분석기 (KoNLPy)
print("[한국어 형태소 분석 - KoNLPy]")
print("=" * 60)

from konlpy.tag import Okt

okt = Okt()

# 형태소 분석 예시
sample = "삼성전자가 3분기 영업이익 10조원을 돌파했다"

print(f"원본: {sample}")
print(f"\n형태소 분석: {okt.morphs(sample)}")
print(f"명사 추출: {okt.nouns(sample)}")
print(f"품사 태깅: {okt.pos(sample)}")

# 금융 텍스트에서 명사 추출
print("\n[금융 텍스트 명사 추출]")
for text in sample_texts[:3]:
    nouns = okt.nouns(text)
    print(f"텍스트: {text}")
    print(f"명사: {nouns}")
    print()

[한국어 형태소 분석 - KoNLPy]
원본: 삼성전자가 3분기 영업이익 10조원을 돌파했다

형태소 분석: ['삼', '성', '전자', '가', '3분', '기', '영업', '이익', '10조원', '을', '돌파', '했다']
명사 추출: ['전자', '영업', '이익', '돌파']
품사 태깅: [('삼', 'Modifier'), ('성', 'Modifier'), ('전자', 'Noun'), ('가', 'Josa'), ('3분', 'Number'), ('기', 'Foreign'), ('영업', 'Noun'), ('이익', 'Noun'), ('10조원', 'Number'), ('을', 'Foreign'), ('돌파', 'Noun'), ('했다', 'Verb')]

[금융 텍스트 명사 추출]
텍스트: 삼성전자, 3분기 실적 발표 영업이익 10조원 돌파
명사: ['삼성', '전자', '실적', '발표', '영업', '이익', '돌파']

텍스트: 코스피 상승세 지속, 외국인 순매수 2000억원
명사: ['코스피', '상승세', '지속', '외국인', '매수']

텍스트: 미국 연준 금리 0.25%p 인상 결정
명사: ['미국', '준', '금리', '인상', '결정']



### 2.3 tiktoken - GPT 토크나이저 (BPE)

**tiktoken**은 OpenAI에서 개발한 BPE(Byte Pair Encoding) 기반 토크나이저입니다.
GPT 모델들이 사용하는 것과 동일한 토크나이징 방식을 제공합니다.

| 인코더 | 사용 모델 |
|--------|-----------|
| cl100k_base | GPT-4, GPT-3.5-turbo, text-embedding-ada-002 |
| p50k_base | GPT-3, Codex |
| o200k_base | GPT-4o |

In [8]:
import tiktoken

# cl100k_base 인코더 사용 (GPT-4와 동일)
encoding = tiktoken.get_encoding("cl100k_base")

print("[tiktoken - GPT 토크나이저]")
print("=" * 60)

# 영어 문장 토큰화
sentences_E = [
    'I love my dog',
    '나는 내 개를 사랑해',
]

print("[영어 텍스트 토큰화]")
for sentence in sentences_E:
    tokens = encoding.encode(sentence)
    decoded = encoding.decode(tokens)
    print(f"문장: {sentence}")
    print(f"토큰 ID: {tokens}")
    print(f"토큰 수: {len(tokens)}")
    print()

[tiktoken - GPT 토크나이저]
[영어 텍스트 토큰화]
문장: I love my dog
토큰 ID: [40, 3021, 856, 5679]
토큰 수: 4

문장: 나는 내 개를 사랑해
토큰 ID: [61415, 16969, 67236, 74623, 18918, 33229, 39519, 239, 34983]
토큰 수: 9



In [9]:
# 토큰화 방식 비교
print("[토큰화 방식 비교]")
print("=" * 60)

test_text = "삼성전자가 3분기 영업이익 10조원을 돌파했다"

print(f"원본: {test_text}")
print()

# 1. 공백 기반
space_tokens = simple_tokenize(test_text)
print(f"공백 기반: {space_tokens} ({len(space_tokens)}개)")

# 2. KoNLPy
try:
    okt_tokens = okt.morphs(test_text)
    print(f"KoNLPy(Okt): {okt_tokens} ({len(okt_tokens)}개)")
except:
    print("KoNLPy: (설치 필요)")

# 3. tiktoken
tik_tokens = encoding.encode(test_text)
print(f"tiktoken: {tik_tokens} ({len(tik_tokens)}개)")

[토큰화 방식 비교]
원본: 삼성전자가 3분기 영업이익 10조원을 돌파했다

공백 기반: ['삼성전자가', '3분기', '영업이익', '10조원을', '돌파했다'] (5개)
KoNLPy(Okt): ['삼', '성', '전자', '가', '3분', '기', '영업', '이익', '10조원', '을', '돌파', '했다'] (12개)
tiktoken: [80690, 120, 33931, 66965, 26799, 20565, 220, 18, 80816, 21121, 39623, 223, 13879, 227, 13094, 6026, 113, 220, 605, 93917, 55421, 18359, 65905, 234, 67218, 234, 169, 244, 20541] (29개)


---
## 3. TF-IDF

### 정의
**TF-IDF (Term Frequency - Inverse Document Frequency)**
- 단어의 중요도를 수치로 표현
- 문서에서 자주 등장하지만, 다른 문서에서는 드문 단어에 높은 가중치

### 수식
- **TF**: 해당 문서에서 단어 등장 빈도
- **IDF**: 전체 문서 중 해당 단어가 등장한 문서의 역빈도
- **TF-IDF = TF × IDF**

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 예시 문서
documents = [
    "삼성전자 반도체 실적 호조",
    "반도체 시장 성장세 지속",
    "삼성전자 스마트폰 점유율 상승",
    "애플 아이폰 신제품 출시",
    "삼성전자 애플 경쟁 심화"
]

print("[TF-IDF 분석]")
print("=" * 60)

# TF-IDF 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(documents)

# 결과를 DataFrame으로
feature_names = tfidf.get_feature_names_out()
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

print("TF-IDF 행렬:")
display(df_tfidf.round(2))

[TF-IDF 분석]
TF-IDF 행렬:


,경쟁,반도체,삼성전자,상승,성장세,스마트폰,시장,신제품,실적,심화,아이폰,애플,점유율,지속,출시,호조
0,0.00,0.46,0.38,0.00,0.00,0.00,0.00,0.00,0.57,0.00,0.00,0.00,0.00,0.00,0.00,0.57
1,0.00,0.42,0.00,0.00,0.52,0.00,0.52,0.00,0.00,0.00,0.00,0.00,0.00,0.52,0.00,0.00
2,0.00,0.00,0.36,0.54,0.00,0.54,0.00,0.00,0.00,0.00,0.00,0.00,0.54,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.52,0.00,0.00,0.52,0.42,0.00,0.00,0.52,0.00
4,0.57,0.00,0.38,0.00,0.00,0.00,0.00,0.00,0.00,0.57,0.00,0.46,0.00,0.00,0.00,0.00


### TF-IDF 해석
- 높은 TF-IDF: 해당 문서에서 중요한 단어
- 낮은 TF-IDF: 흔하거나 중요하지 않은 단어
- 금융 활용: 뉴스에서 핵심 키워드 추출

---
## 4. 임베딩과 유사도 검색

### TF-IDF vs 임베딩
| 구분 | TF-IDF | 임베딩 (Word2Vec, BERT 등) |
|------|--------|---------------------------|
| 표현 | 희소 벡터 (Sparse) | 밀집 벡터 (Dense) |
| 의미 | 빈도 기반 | 문맥/의미 기반 |
| 유사도 | 단어 독립 | 의미적 유사도 |

### 임베딩이란?
- 텍스트를 고정 크기의 벡터(숫자 배열)로 변환
- **의미적으로 유사한 텍스트는 유사한 벡터**
- 예: "주가 상승" ↔ "주식 가격 증가" → 유사한 벡터

In [15]:
# 임베딩 모델 로드 (KURE-v1: 한국어 특화 모델)
print("[임베딩 모델 로드]")
print("=" * 60)

from langchain_huggingface import HuggingFaceEmbeddings

# 한국어 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="nlpai-lab/KURE-v1")
print("KURE-v1 모델 로드 완료!")
print("- 한국어에 특화된 문장 임베딩 모델")
print("- 벡터 차원: 1024")

[임베딩 모델 로드]
KURE-v1 모델 로드 완료!
- 한국어에 특화된 문장 임베딩 모델
- 벡터 차원: 1024


In [20]:
# 문장 임베딩 생성
print("[문장 임베딩 생성]")
print("=" * 60)

# 금융 관련 문장들
finance_sentences = [
    "삼성전자 주가가 상승했습니다",
    "SK하이닉스 주가 상승"
]

# 각 문장을 벡터로 변환
vectors = []
for sentence in finance_sentences:
    vec = embeddings.embed_query(sentence)
    vectors.append(vec)
    print(f"문장: {sentence}")
    print(f"벡터 길이: {len(vec)}")
    print(f"벡터 일부: {vec[:5]}...")
    print()

[문장 임베딩 생성]
문장: 삼성전자 주가가 상승했습니다
벡터 길이: 1024
벡터 일부: [-0.02644462324678898, -0.01891239732503891, -0.03149032220244408, 0.03106190264225006, -0.002300356747582555]...

문장: SK하이닉스 주가 상승
벡터 길이: 1024
벡터 일부: [-0.044104091823101044, 0.022867834195494652, -0.042592231184244156, 0.05487203225493431, -0.012192251160740852]...

[문장 간 벡터 유사도]
문장 1: 삼성전자 주가가 상승했습니다
문장 2: SK하이닉스 주가 상승
코사인 유사도: 0.7218


### TF-IDF vs Embedding 유사도 비교

In [21]:
# TF-IDF vs Embedding 비교
from sklearn.metrics.pairwise import cosine_similarity as sk_cosine

# 쿼리와 비교 문장
query = "삼성전자 주가가 상승했습니다"
sentences = [
    "삼성전자 주식 가격이 올랐습니다",  # 의미 동일, 단어 다름
    "코스피 지수가 하락했습니다",
    "오늘 날씨가 좋습니다",
]

print(f"쿼리: '{query}'")
print()

# ---- TF-IDF 유사도 ----
tfidf_vec = TfidfVectorizer()
tfidf_matrix = tfidf_vec.fit_transform([query] + sentences)
tfidf_sim = sk_cosine(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

print("[TF-IDF] 단어 겹침 기반")
for s, sim in zip(sentences, tfidf_sim):
    print(f"  {sim:.3f} | {s}")

# ---- Embedding 유사도 ----
query_vec = embeddings.embed_query(query)
emb_sim = [sk_cosine([query_vec], [embeddings.embed_query(s)])[0][0] for s in sentences]

print()
print("[Embedding] 의미 기반")
for s, sim in zip(sentences, emb_sim):
    print(f"  {sim:.3f} | {s}")

# ---- 핵심 비교 ----
print()
print("[결론]")
print(f"'주가가 상승' vs '주식 가격이 올랐습니다'")
print(f"  TF-IDF:    {tfidf_sim[0]:.3f} (단어 다름 → 낮음)")
print(f"  Embedding: {emb_sim[0]:.3f} (의미 같음 → 높음)")

[TF-IDF vs Embedding 유사도 비교]
쿼리: '삼성전자 주가가 상승했습니다'

[TF-IDF 유사도] (단어 빈도 기반)
  [0.209] 삼성전자 주식 가격이 올랐습니다
  [0.000] 코스피 지수가 하락했습니다
  [0.000] 오늘 날씨가 좋습니다
  [0.000] SK하이닉스 반도체 실적 호조

→ TF-IDF 1위: '삼성전자 주식 가격이 올랐습니다'

[Embedding 유사도] (의미 기반)
  [0.913] 삼성전자 주식 가격이 올랐습니다
  [0.544] 코스피 지수가 하락했습니다
  [0.490] 오늘 날씨가 좋습니다
  [0.565] SK하이닉스 반도체 실적 호조

→ Embedding 1위: '삼성전자 주식 가격이 올랐습니다'

[비교 결과]
TF-IDF 1위:    '삼성전자 주식 가격이 올랐습니다'
Embedding 1위: '삼성전자 주식 가격이 올랐습니다'

→ 두 방법의 결과가 동일합니다.

[핵심 차이]
'삼성전자 주가가 상승했습니다' vs '삼성전자 주식 가격이 올랐습니다'
- TF-IDF: 단어가 완전히 같지 않으면 유사도가 낮을 수 있음
- Embedding: 의미가 비슷하면 유사도가 높게 나올 수 있음


---
## 학습 정리

### 1. NLP 파이프라인
```
텍스트 → 토큰화 → 벡터화(TF-IDF/임베딩) → 분석/검색
```

### 2. 토큰화 방법
```python
# KoNLPy (한국어 형태소 분석)
from konlpy.tag import Okt
okt = Okt()
tokens = okt.morphs(text)  # 형태소 분석
nouns = okt.nouns(text)    # 명사 추출

# tiktoken (GPT 토크나이저)
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
tokens = encoding.encode(text)
```

### 3. TF-IDF
```python
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
matrix = tfidf.fit_transform(documents)
```

### 4. 임베딩과 유사도 검색
```python
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="nlpai-lab/KURE-v1")

# 문장 벡터화
vector = embeddings.embed_query(text)

# 유사도 계산
similarity = cosine_similarity([query_vec], [doc_vec])
```

### 5. 텍스트 표현 방법 비교
| 방법 | 특징 | 활용 |
|------|------|------|
| TF-IDF | 빈도 기반, 희소 | 키워드 추출, 문서 분류 |
| 임베딩 | 의미 기반, 밀집 | 유사도 검색, 감성 분석 |

---

### 다음 차시 예고
- 26차시: [실습] 금융 뉴스 감성 분석
  - 네이버 금융 뉴스 크롤링 (모듈 2 연계)
  - 감성 사전 기반 분석
  - 뉴스 감성과 주가 관계